In [1]:
# import necessary Libraries
import os
from PIL import Image
import numpy as np
import sys

In [2]:
# Read folder for aquarium fish species label
folders = next(os.walk('dataset/train'))[1]

In [3]:
# print folder 
print(folders)

['Arowana', 'Crowntail Betta', 'Flowerhorn', 'Guppy', 'Halfmoon Betta', 'Oscar', 'Ryukin GoldFish']


In [4]:
#read all images in train folder 
X = np.zeros((1177,224,224,3), dtype=np.int8)
Y = []
file_count = 0
folder_count = 0
for folder in folders:
    files = os.listdir('dataset/train/' + folder)
    for file in files:
        image = Image.open('dataset/train/' + folder + '/' + file)
        x_image = np.array(image)
        X[file_count,:,:] = x_image
        image.close()
        Y.append(folder_count)
        file_count += 1
    folder_count += 1

In [5]:
#show how mamny image sample exists in train folder and print  
X = np.array(X)
Y = np.array(Y)
print(X.shape)
print(Y.shape)

(1177, 224, 224, 3)
(1177,)


In [6]:
# how many folder are in train folder and print label 
print(Y)

[0 0 0 ... 6 6 6]


In [7]:
# import sklearn to split datasets , 20% test data and 80% train data 
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.20, random_state=42)

In [8]:
#view how many data are allocated for train and test 
print(X_train.shape)
print(Y_train.shape)
print(X_test.shape)
print(Y_test.shape)

(941, 224, 224, 3)
(941,)
(236, 224, 224, 3)
(236,)


In [9]:
# import keras library for vectorized and model development 

import keras
from keras.models import Sequential,Input,Model
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import BatchNormalization
from keras.layers.advanced_activations import LeakyReLU
from keras import backend as K
import tensorflowjs as tfjs

# setup configuration of model
batch_size = 64
num_classes = 7
epochs = 15

# input image dimensions
img_rows, img_cols = 224, 224

from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.20, random_state=42)

print(X_train.shape)
print(Y_train.shape)
print(X_test.shape)
print(Y_test.shape)


#Reshaping Train and test data
X_train = X_train.reshape(-1, 224,224, 3)
X_test = X_test.reshape(-1, 224,224, 3)


X_train = X_train.astype('float32')
X_test = X_test.astype('float32')

X_train = X_train / 255.
X_test = X_test / 255.


print('X_train shape:', X_train.shape)
print('X_test shape', X_test.shape)
print(X_train.shape[0], 'train samples')
print(X_test.shape[0], 'test samples')

(941, 224, 224, 3)
(941,)
(236, 224, 224, 3)
(236,)
X_train shape: (941, 224, 224, 3)
X_test shape (236, 224, 224, 3)
941 train samples
236 test samples


In [10]:
# convert class vectors to binary class matrices
from keras import utils as np_utils

Y_train = keras.utils.np_utils.to_categorical(Y_train)
Y_test = keras.utils.np_utils.to_categorical(Y_test)

# setup CNN model configuration 
model = Sequential([
    Conv2D(32, kernel_size=(3, 3),activation='linear',padding='same',input_shape=(224,224,3)),
    LeakyReLU(alpha=0.1),
    MaxPooling2D((2, 2),padding='same'),
    Dropout(0.1),
    
    Conv2D(64, (3, 3), activation='linear',padding='same'),
    LeakyReLU(alpha=0.1),
    MaxPooling2D(pool_size=(2, 2),padding='same'),
    Dropout(0.1),
    
    Conv2D(128, (3, 3), activation='linear',padding='same'),
    LeakyReLU(alpha=0.1),               
    MaxPooling2D(pool_size=(2, 2),padding='same'),
    Dropout(0.1),
    
    Conv2D(256, (3, 3), activation='linear',padding='same'),
    LeakyReLU(alpha=0.1),               
    MaxPooling2D(pool_size=(2, 2),padding='same'),
    Dropout(0.1),
    
    Flatten(),
    Dense(128, activation='linear'),
    LeakyReLU(alpha=0.1),         
    Dropout(0.1),
    Dense(num_classes, activation='softmax')
])

In [11]:
# With data augmentation to prevent overfitting (accuracy 0.99286)
from keras.preprocessing.image import ImageDataGenerator


datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        rotation_range=10,  # randomly rotate images in the range (degrees, 0 to 180)
        zoom_range = 0.1, # Randomly zoom image 
        width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=False,  # randomly flip images
        vertical_flip=False)  # randomly flip images


datagen.fit(X_train)

In [12]:
# configure backpropragation by provide loss function and optimizer
import tensorflow
model.compile(loss=keras.losses.categorical_crossentropy, optimizer=tensorflow.keras.optimizers.Adam(),metrics=['accuracy'])

In [13]:
# display model configuration correctly setup or not 
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 224, 224, 32)      896       
                                                                 
 leaky_re_lu (LeakyReLU)     (None, 224, 224, 32)      0         
                                                                 
 max_pooling2d (MaxPooling2D  (None, 112, 112, 32)     0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 112, 112, 32)      0         
                                                                 
 conv2d_1 (Conv2D)           (None, 112, 112, 64)      18496     
                                                                 
 leaky_re_lu_1 (LeakyReLU)   (None, 112, 112, 64)      0         
                                                        

In [14]:
# setup training 
model.fit(X_train, Y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(X_test, Y_test))

Epoch 1/15
15/15 [==============================] - 86s 5s/step - loss: 1.9954 - accuracy: 0.3560 - val_loss: 1.3867 - val_accuracy: 0.5085
Epoch 2/15
15/15 [==============================] - 77s 5s/step - loss: 1.0830 - accuracy: 0.6079 - val_loss: 0.8546 - val_accuracy: 0.6695
Epoch 3/15
15/15 [==============================] - 65s 4s/step - loss: 0.7148 - accuracy: 0.7460 - val_loss: 0.5805 - val_accuracy: 0.8093
Epoch 4/15
15/15 [==============================] - 64s 4s/step - loss: 0.4964 - accuracy: 0.8225 - val_loss: 0.5100 - val_accuracy: 0.8390
Epoch 5/15
15/15 [==============================] - 65s 4s/step - loss: 0.3367 - accuracy: 0.8746 - val_loss: 0.4596 - val_accuracy: 0.8305
Epoch 6/15
15/15 [==============================] - 63s 4s/step - loss: 0.2349 - accuracy: 0.9182 - val_loss: 0.4194 - val_accuracy: 0.8644
Epoch 7/15
15/15 [==============================] - 63s 4s/step - loss: 0.1203 - accuracy: 0.9660 - val_loss: 0.3876 - val_accuracy: 0.8729
Epoch 8/15
15/15 [==

In [15]:
#Save Everything Including Architecture,Weights, necessary Bias and optimizer and loss functions of whole network
model.save('FinalSave.h5')

In [16]:
from keras.models import load_model
Final = load_model('FinalSave.h5')

score = Final.evaluate(X_train, Y_train, verbose=1)
print('Train loss:', score[0])
print('Train accuracy:', score[1])

Final.save('AquariumFish.h5')

30/30 [==============================] - 12s 407ms/step - loss: 0.0704 - accuracy: 0.9766
Train loss: 0.0704299807548523
Train accuracy: 0.976620614528656


In [17]:
Final.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 224, 224, 32)      896       
                                                                 
 leaky_re_lu (LeakyReLU)     (None, 224, 224, 32)      0         
                                                                 
 max_pooling2d (MaxPooling2D  (None, 112, 112, 32)     0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 112, 112, 32)      0         
                                                                 
 conv2d_1 (Conv2D)           (None, 112, 112, 64)      18496     
                                                                 
 leaky_re_lu_1 (LeakyReLU)   (None, 112, 112, 64)      0         
                                                        

In [18]:
#Via Save Weights and use same Network Architecture 
import keras
Final.save_weights('FinalSaveWeights.h5')

In [19]:
import keras
from keras.models import Sequential,Input,Model
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import BatchNormalization
from keras.layers.advanced_activations import LeakyReLU
from keras import backend as K
import tensorflowjs as tfjs

FinalModel2 = Sequential([
    Conv2D(32, kernel_size=(3, 3),activation='linear',padding='same',input_shape=(224,224,3)),
    LeakyReLU(alpha=0.1),
    MaxPooling2D((2, 2),padding='same'),
    Dropout(0.1),
    
    Conv2D(64, (3, 3), activation='linear',padding='same'),
    LeakyReLU(alpha=0.1),
    MaxPooling2D(pool_size=(2, 2),padding='same'),
    Dropout(0.1),
    
    Conv2D(128, (3, 3), activation='linear',padding='same'),
    LeakyReLU(alpha=0.1),               
    MaxPooling2D(pool_size=(2, 2),padding='same'),
    Dropout(0.1),
    
    Conv2D(256, (3, 3), activation='linear',padding='same'),
    LeakyReLU(alpha=0.1),               
    MaxPooling2D(pool_size=(2, 2),padding='same'),
    Dropout(0.1),
    
    Flatten(),
    Dense(128, activation='linear'),
    LeakyReLU(alpha=0.1),         
    Dropout(0.1),
    Dense(7, activation='softmax')
])

In [20]:
from tensorflow import keras
FinalModel2.load_weights('FinalSaveWeights.h5')
FinalModel2.compile(loss=keras.losses.categorical_crossentropy, optimizer=keras.optimizers.Adam(),metrics=['accuracy'])

In [21]:
score = FinalModel2.evaluate(X_train, Y_train, verbose=0)
print('Train loss:', score[0])
print('Train accuracy:', score[1])

FinalModel2.save('AquariumFish.h5')

Train loss: 0.0704299807548523
Train accuracy: 0.976620614528656


In [22]:
Final.get_weights()

[array([[[[-1.00301377e-01,  1.17454149e-01,  1.32522788e-02,
           -1.21867321e-01,  6.23168722e-02,  1.30472973e-01,
            1.61701832e-02,  4.09356393e-02,  1.12773992e-01,
           -9.79727134e-03,  2.09679287e-02, -6.64309561e-02,
            9.47338641e-02,  1.18867867e-01, -1.24415062e-01,
            1.30459458e-01, -4.77645248e-02, -1.38848290e-01,
            2.97172880e-03,  1.20002404e-01,  1.11230686e-01,
            1.03574945e-02,  8.40345919e-02,  8.23646188e-02,
            7.75989369e-02, -1.34676278e-01, -6.77391291e-02,
           -1.01290844e-01,  2.59987433e-02,  7.39823952e-02,
            6.72197416e-02,  1.24278083e-01],
          [ 1.17277712e-01,  1.19619153e-01,  5.34085967e-02,
           -4.39637676e-02, -6.87569305e-02,  4.43519093e-02,
            1.01235978e-01,  7.36566074e-03, -1.45043954e-01,
            1.76904332e-02, -7.86590502e-02, -1.02101393e-01,
           -3.43136936e-02, -9.42980945e-02,  6.61863610e-02,
            1.02524303e-

In [23]:
Final.optimizer

In [24]:
score = Final.evaluate(X_test, Y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Test loss: 0.7983858585357666
Test accuracy: 0.8644067645072937


In [25]:
score = Final.evaluate(X_train, Y_train, verbose=1)
print('Train loss:', score[0])
print('Train accuracy:', score[1])

30/30 [==============================] - 13s 432ms/step - loss: 0.0704 - accuracy: 0.9766
Train loss: 0.0704299807548523
Train accuracy: 0.976620614528656


In [26]:
tfjs.converters.save_keras_model(model,"./ModelSaveinTensorflowjs")

In [27]:
#Save JSON is only save architecture of Model
json_string = Final.to_json()
json_string

'{"class_name": "Sequential", "config": {"name": "sequential", "layers": [{"class_name": "InputLayer", "config": {"batch_input_shape": [null, 224, 224, 3], "dtype": "float32", "sparse": false, "ragged": false, "name": "conv2d_input"}}, {"class_name": "Conv2D", "config": {"name": "conv2d", "trainable": true, "batch_input_shape": [null, 224, 224, 3], "dtype": "float32", "filters": 32, "kernel_size": [3, 3], "strides": [1, 1], "padding": "same", "data_format": "channels_last", "dilation_rate": [1, 1], "groups": 1, "activation": "linear", "use_bias": true, "kernel_initializer": {"class_name": "GlorotUniform", "config": {"seed": null}}, "bias_initializer": {"class_name": "Zeros", "config": {}}, "kernel_regularizer": null, "bias_regularizer": null, "activity_regularizer": null, "kernel_constraint": null, "bias_constraint": null}}, {"class_name": "LeakyReLU", "config": {"name": "leaky_re_lu", "trainable": true, "dtype": "float32", "alpha": 0.10000000149011612}}, {"class_name": "MaxPooling2D",

In [28]:
from keras.models import model_from_json
Final_architecture = model_from_json(json_string)
Final_architecture.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 224, 224, 32)      896       
                                                                 
 leaky_re_lu (LeakyReLU)     (None, 224, 224, 32)      0         
                                                                 
 max_pooling2d (MaxPooling2D  (None, 112, 112, 32)     0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 112, 112, 32)      0         
                                                                 
 conv2d_1 (Conv2D)           (None, 112, 112, 64)      18496     
                                                                 
 leaky_re_lu_1 (LeakyReLU)   (None, 112, 112, 64)      0         
                                                        